Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
    - speedup
    - eficiencia
 - output: 
    - p_cores


In [126]:
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

In [154]:
df_clean = pd.read_csv("dataset/clean_data.csv")

# df_clean['cores'] = df_clean['cores']/df_clean['cores'].max()
df_clean['input'] = df_clean['input']/df_clean['input'].max()

In [155]:
X = df_clean[['input', 'average_runtime', 'speedup', 'efficiency']].values
y = df_clean['cores'].values

### Regression Models

#### MLP Regression Model

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [160]:
learning_rate = 0.001
input_shape = (X_train.shape[1],)

callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape = input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, batch_size=32, epochs=500, verbose=2, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/500
7/7 - 0s - loss: 389.1112 - val_loss: 395.7542 - 456ms/epoch - 65ms/step
Epoch 2/500
7/7 - 0s - loss: 382.4682 - val_loss: 390.4314 - 28ms/epoch - 4ms/step
Epoch 3/500
7/7 - 0s - loss: 377.3694 - val_loss: 385.2799 - 27ms/epoch - 4ms/step
Epoch 4/500
7/7 - 0s - loss: 373.3969 - val_loss: 380.0065 - 28ms/epoch - 4ms/step
Epoch 5/500
7/7 - 0s - loss: 367.9297 - val_loss: 374.9648 - 28ms/epoch - 4ms/step
Epoch 6/500
7/7 - 0s - loss: 362.7386 - val_loss: 369.5460 - 27ms/epoch - 4ms/step
Epoch 7/500
7/7 - 0s - loss: 357.4028 - val_loss: 363.7054 - 27ms/epoch - 4ms/step
Epoch 8/500
7/7 - 0s - loss: 350.3540 - val_loss: 356.1862 - 28ms/epoch - 4ms/step
Epoch 9/500
7/7 - 0s - loss: 342.0305 - val_loss: 347.7037 - 30ms/epoch - 4ms/step
Epoch 10/500
7/7 - 0s - loss: 333.7207 - val_loss: 338.2690 - 32ms/epoch - 5ms/step
Epoch 11/500
7/7 - 0s - loss: 324.5290 - val_loss: 327.6654 - 29ms/epoch - 4ms/step
Epoch 12/500
7/7 - 0s - loss: 314.1601 - val_loss: 316.7049 - 30ms/epoch - 4ms/step

In [161]:
loss = model.evaluate(X_test, y_test)
print(f'Erro no conjunto de teste: {loss}')

predictions = model.predict(X_test)

for i in range(10):
    print(f'Valor Real: {y_test[i]}, Previsão: {predictions[i][0]}')

3/3 [==============================] - 0s 2ms/step - loss: 5.5731
Erro no conjunto de teste: 5.573118686676025
3/3 [==============================] - 0s 1ms/step
Valor Real: 6, Previsão: 6.2731828689575195
Valor Real: 29, Previsão: 29.967500686645508
Valor Real: 19, Previsão: 20.83418846130371
Valor Real: 23, Previsão: 21.895061492919922
Valor Real: 27, Previsão: 27.277957916259766
Valor Real: 6, Previsão: 5.2311906814575195
Valor Real: 2, Previsão: 1.262270212173462
Valor Real: 1, Previsão: -2.239701271057129
Valor Real: 18, Previsão: 18.272544860839844
Valor Real: 31, Previsão: 31.510345458984375


#### MLP Regression Model with Leave-One-Out-Validation (LOOCV)

In [156]:
loo = LeaveOneOut()
mse_scores = []
callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))


for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    learning_rate = 0.001
    input_shape = (X_train.shape[1],)

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=100, verbose=0, callbacks=[callback])

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

print(f'Erro médio quadrático usando LOOCV: {min(mse_scores)}')

1/1 [==============================] - 0s 37ms/step
Erro médio quadrático usando LOOCV: 9.259392754756846e-07


In [158]:
x_test_new = [
    [1,0.24561333656311035,1.0264687524571021,0.03207714851428444],
    [1,0.23585212230682373,1.0689512254665345,0.042758049018661384]
]

model.predict(x=x_test_new)

1/1 [==============================] - 0s 29ms/step


array([[47.905838],
       [43.49272 ]], dtype=float32)